In [1]:
from pystac_client import Client

# Connect to Landsat STAC catalog
catalog = Client.open('https://landsatlook.usgs.gov/stac-server')

# Search for one scene
search = catalog.search(
    collections=['landsat-c2l2-sr'],
    bbox=[-122.5, 37.7, -122.3, 37.9],  # San Francisco
    datetime='2023-06-01/2023-08-31',
    limit=1
)

items = list(search.items())
print(f"Found {len(items)} scene(s)")
print(f"Scene ID: {items[0].id if items else 'None'}")

Found 42 scene(s)
Scene ID: LC09_L2SP_044034_20230826_20230828_02_T1_SR


In [6]:
item = items[0]
print("Assets:")
for asset_key, asset in item.assets.items():
    print(f" - {asset_key}: {asset.href}")

Assets:
 - thumbnail: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2023/044/034/LC09_L2SP_044034_20230826_20230828_02_T1/LC09_L2SP_044034_20230826_20230828_02_T1_thumb_small.jpeg
 - reduced_resolution_browse: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2023/044/034/LC09_L2SP_044034_20230826_20230828_02_T1/LC09_L2SP_044034_20230826_20230828_02_T1_thumb_large.jpeg
 - index: https://landsatlook.usgs.gov/stac-browser/collection02/level-2/standard/oli-tirs/2023/044/034/LC09_L2SP_044034_20230826_20230828_02_T1/LC09_L2SP_044034_20230826_20230828_02_T1
 - MTL.json: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2023/044/034/LC09_L2SP_044034_20230826_20230828_02_T1/LC09_L2SP_044034_20230826_20230828_02_T1_MTL.json
 - coastal: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2023/044/034/LC09_L2SP_044034_20230826_20230828_02_T1/LC09_L2SP_044034_20230826_20230828_02_T1_SR_B1.TIF
 - blue: htt